# Book Analysis

This notebook explores a dataset of book details scraped from [Tor](https://publishing.tor.com/).

## Imports

In [ ]:
from os import environ

from dotenv import load_dotenv
import pandas as pd
import altair as alt

## Setup

In [ ]:
load_dotenv()

pd.set_option("max_colwidth", 150)  # Display more text

## Data sourcing

In [ ]:
def string_to_list(string: str) -> list[str]:
    """Returns a list from a list stored as a string."""
    return string[1:-1].split(", ")

In [ ]:
books = pd.read_csv(environ["FINAL_BOOK_FILEPATH"], parse_dates=["publication_date"],
                    converters={"format": string_to_list, "contributors": string_to_list})

In [ ]:
books.sample(3)

## Data exploration

**How many books are missing descriptions or page numbers? Can you work out why?**

**How many books are not part of a series**?

**What is the name of the longest series?**

**How many books were published in each month of the year (bar chart)?**

**What's the average number of pages?**

**What proportion of books have more than one author (pie chart)?**

**How many books were published each year (line chart)?**